## What is an ORM?

### First describe what object-oriented programming is
- object-oriented programming is a programming concept based on the concept of "objects"
- "objects" contain data, known as "fields" or "attributes", and code specific to them, known as "procedures" or "methods"
- One example would be a Car object
    - fields: {number of wheels, gas mileage, color}
    - methods: {change_color(), get_gas_mileage(), add_wheels()}

### Compare object-oriented programming to SQL
- SQL databases can only store and manipulate scalar values (integers and strings), and not objects
- To load data into a database the values must be converted to simple types
- To extract data from the database, they can only be extracted as simple values and then combined afterwards to make an object

### How ORM solves our problem
- ORM stands for Object-Relational Mapping
- ORM converts our objects into a form that can be stored into the database, and extracted from
- It also abstracts the conversion from us, by providing us simple code for doing so
- Ultimately reduces the amount of code and complexity of it

### peewee package
- currently used at imedicare
- python package that provides a small and simple ORM

## peewee tutorial
4 step process:
- (1) Installation
- (2) Model Definition
- (3) Storing
- (4) Retrieving

### Installation
- Options to install:
    - (1) 
          pip install peewee
    - (2) 
          git clone https://github.com/coleifer/peewee.git
          cd peewee
          python setup.py install

- Any issues go to this link first https://stackoverflow.com/questions/2817869/error-unable-to-find-vcvarsall-bat 

### Model Definition

In [1]:
# import * --> imports all functions and variables from the peewee package
# all variables not previously defined come from the peewee class
from peewee import *

from datetime import date

In [2]:
# SqliteDatabase is a lightweight disk-based database that doesn't require separate server
# comes from peewee package
db = SqliteDatabase('people.db')

# Model comes from the peewee package
class Person(Model):
    '''
    3 different data types, all which are provided by the peewee class
    '''
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()
    
    # Meta class required for every defined object to tell which database the object will use
    class Meta:
        database = db  # this class uses the 'people.db' database

In [3]:
# 'Pet' class uses owner variable to reference the 'Person' object
class Pet(Model):
    owner = ForeignKeyField(Person, related_name='pets') # reference to previous object 'Person'
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db # this model uses the "people.db" database

In [4]:
# connect to the database
db.connect()

In [5]:
# create tables in the database - using our previusly defined objects above
# try and except block --> prevent code from breaking if you already have created the tables
try:
    db.create_tables([Person, Pet])
except OperationalError:
    print ("Tables already exist.")

*** Pause here and check you file directory. You will see a file called 'people' with the type as 'Data Base File' ***

### Storing

In [6]:
bob = Person(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
joe = Person(name='Joe', birthday=date(1960, 1, 15), is_relative=True)

bob.save() # bob is now stored in the database
joe.save() # joe is too

# 1 is the number of rows that have been modified

1

In [7]:
# Now create a 'Pet' object, with the owner as our 'Person' object above 'bob'
bob_cat_1 = Pet(owner=bob, name='Mittens', animal_type='cat')
bob_cat_2 = Pet(owner=bob, name='Spots', animal_type='cat')
bob_cat_3 = Pet(owner=bob, name='Tom', animal_type='cat')

bob_cat_1.save()
bob_cat_2.save()
bob_cat_3.save()

1

In [8]:
# delete the second 'Pet' from the database
bob_cat_3.delete_instance()

1

In [9]:
# assign the owner of 'bob_cat_2' to a different 'Person' 'joe'
bob_cat_2.owner = joe
bob_cat_2.save()

# rename the variable to show the updated owner (not necessary but better naming)
joe_cat_2 = bob_cat_2

### Retrieving

In [10]:
bob_from_db = Person.get(Person.name == 'Bob')
joe_from_db = Person.get(Person.name == 'Joe')

In [11]:
for person in Person.select():
    print (person.name, person.is_relative)

Bob True
Joe True


In [12]:
# join two tables for query
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print (pet.name)

Mittens


In [15]:
# alternative - join two tables for query
for pet in Pet.select().join(Person).where(Pet.owner == bob):
    print (pet.name)

Mittens


In [16]:
# Print all 'Pet' data for each 'Person'
for person in Person.select():
    print (person.name, person.pets.count(), 'pets')
    for pet in person.pets:
        print ('\t', pet.name, pet.animal_type)

Bob 1 pets
	 Mittens cat
Joe 1 pets
	 Spots cat


In [17]:
# close the database
db.close()